<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downsampling</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Взвешивание классов</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Отток клиентов

Из банка стали уходить клиенты. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

В этом проекте спрогнозируем, уйдёт клиент из банка в ближайшее время или нет, так как эта информация поможет маркетологам составить стратегию для сохранения текущих клиентов.  

Для исследования предоставлены исторические данные о поведении 10000 клиентов и расторжении договоров с банком. А именно данные: о регионе проживания, поле, возрасте, наличии кредитной карты, сколько лет человек является клиеном банка, балансе на счете, кредитном рейтинге, количестве исчпользуемых продуктов банка, количестве продуктов банка, используемых клиентом, активности и предполагаемой заработной плате. На основании этих данных будет составлена модель для прогнозирования ухода клиента из банка. 

План работы:
- Загрузить и подготовить данные.
- Исследовать баланс классов, обучить модель без учёта дисбаланса, сделать выводы.
- Улучшить качество модели, учитывая дисбаланс классов. Обучить разные модели и найти лучшую, сделать выводы.
- Провести финальное тестирование.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('datasets/Churn.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Создали датафрейм, изучили общую информацию. В датасете содержатся столбцы:

Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак
- Exited — факт ухода клиента

Целевой признак - категориальный, так как факт ухода клиента может иметь только два значения (клиент ушел или не ушел), значит решается задача бинарной классификации.

Названия столбцов не соответствуют стандартам, изменим их в соответствии со стандартами.

In [4]:
data.columns = data.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]

C:\Users\oyuki\AppData\Local\Temp\ipykernel_17168\3846741158.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.columns = data.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]


In [5]:
data.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

Теперь столбцы имеют названия, соответствующие стандартам.

Также в данных есть пропуски (признак tenure), это мешает работе с датасетом, необходимо их заполнить.

In [6]:
data['tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

Количество лет, в течение которых клиент пользуется услугами банка, варьируется от 0 до 10.

Заполним пропуски специальным значением -1, после чего изменим тип данных столбца на 'object'. Таким образом получим категориальный признак, так как значений в этом признаке ограниченное количество

In [7]:
data['tenure'] = data['tenure'].fillna(-1)
data['tenure'] = data['tenure'].astype('object')

Также удалим из датафрейма столбцы surname, row_number, customer_id, так как фамилия клиента и индексы никак не влияют на целевой признак и могут только запутать модель.

In [8]:
useless = ['surname', 'row_number', 'customer_id']
data = data.drop(useless, axis=1)

Также в данных есть категориальные признаки, это помешает обучению моделей. Преобразуем эти признаки в численные с помощью прямого кодирования OHE (One-Hot Encoding).

In [9]:
data_ohe = pd.get_dummies(data, drop_first=True)

Далее нужно разделить датасет на признаки и целевой признак, а также на обучающую (60%), валидационную (20%) и тестовую (20%) выборки.

In [10]:
features = data_ohe.drop('exited', axis=1)
target = data_ohe['exited']

features_train, features_valid_and_test, target_train, target_valid_and_test = train_test_split(features, target, 
                                                                                                test_size=0.4, 
                                                                                                random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid_and_test, target_valid_and_test,
                                                                           test_size=0.5, random_state=12345)

Удостоверимся, что данные поделены верно.

In [11]:
features_train.shape

(6000, 21)

In [12]:
features_valid.shape

(2000, 21)

In [13]:
features_test.shape

(2000, 21)

In [14]:
target_train.shape

(6000,)

In [15]:
target_valid.shape

(2000,)

In [16]:
target_test.shape

(2000,)

Теперь данные разделены на три выборки: обучающая, валидационная и тестовая.

В данных могут быть количественные признаки с разными разбросами значений. Чтобы алгоритм обучения не решил, что какой-то признак важнее другого, нужно масштабировать данные, сделаем это с помощью стандартизации данных.

Выделим столбцы с количественными признаками.

In [17]:
numeric = ['credit_score', 'age', 'balance', 'num_of_products', 'estimated_salary']

In [18]:
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [19]:
data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_score       10000 non-null  int64  
 1   age                10000 non-null  int64  
 2   balance            10000 non-null  float64
 3   num_of_products    10000 non-null  int64  
 4   has_cr_card        10000 non-null  int64  
 5   is_active_member   10000 non-null  int64  
 6   estimated_salary   10000 non-null  float64
 7   exited             10000 non-null  int64  
 8   geography_Germany  10000 non-null  uint8  
 9   geography_Spain    10000 non-null  uint8  
 10  gender_Male        10000 non-null  uint8  
 11  tenure_0.0         10000 non-null  uint8  
 12  tenure_1.0         10000 non-null  uint8  
 13  tenure_2.0         10000 non-null  uint8  
 14  tenure_3.0         10000 non-null  uint8  
 15  tenure_4.0         10000 non-null  uint8  
 16  tenure_5.0         1000

### Вывод

Итак, данные подготовлены к работе. Что было сделано:

1. Сначала был создан датафрейм, изучен.
2. Обработаны пропуски в столбце *tenure*, заполнены медианным значением.
3. Удален столбце *surname*, так как он не нужен для решения поставленной задачи.
4. Категориальные признаки превращены в численные с помощью прямого кодирования OHE.
5. Датасет разделен на обучающую, валидационную и тестовую выборки.
6. Столбцы с количественными признаками масштабированы, чтобы избежать того, что модель выделит какие-то признаки как более важные.

## Исследование задачи

Посмотрим, есть ли дисбаланс классов в данных.

In [20]:
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Дисбаланс определенно есть, соотношение далеко от 1:1. В данном случае около 80% и 20%.

Сначала обучим и исследуем модели без учета дисбаланса классов.

Начнем с модели случайного леса, подберем параметры с помощью GridSearch.

In [21]:
parameters_forest = { 'n_estimators': range (1, 81, 10),
                     'max_depth': range (2,15,2),
                     'min_samples_leaf': range (2,11,2),
                     'min_samples_split': range (2,11,2) }

In [22]:
parameters_tree = { 'max_depth': range (2,15,2),
                   'min_samples_leaf': range (2,11,2),
                   'min_samples_split': range (2,11,2) }

In [23]:
%%time

rfc_unbalanced = RandomForestClassifier(random_state=12345)

rfc_gs_unbalanced = GridSearchCV(estimator = rfc_unbalanced, param_grid = parameters_forest, cv=5, scoring='f1', n_jobs = -1)
rfc_gs_unbalanced.fit(features_train, target_train)
rfc_gs_unbalanced.best_params_

CPU times: total: 49.5 s
Wall time: 7min 20s


{'max_depth': 14,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 31}

In [24]:
rfc_unbalanced = rfc_gs_unbalanced.best_estimator_
rfc_unbalanced.fit(features_train, target_train)

predictions_valid = rfc_unbalanced.predict(features_valid)
predicted_proba_valid = rfc_unbalanced.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели случайного леса:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели случайного леса: 0.5557299843014128
AUC-ROC метрика: 0.8503348072514351


Итак, получили результат f1 0.56, не очень высокий, но это предварительная модель и предварительная оценка. Метрика качества AUC-ROC же 0.843, что является неплохим результатом (значительно больше чем 0.5 у случайной модели).

Изучим модель решающего дерева, подберем параметры.

In [25]:
%%time

dtc_unbalanced = DecisionTreeClassifier(random_state=12345)

dtc_gs_unbalanced = GridSearchCV(estimator = dtc_unbalanced, param_grid = parameters_tree, cv=5, scoring='f1', n_jobs = -1)
dtc_gs_unbalanced.fit(features_train, target_train)
dtc_gs_unbalanced.best_params_

CPU times: total: 2.77 s
Wall time: 8.54 s


{'max_depth': 12, 'min_samples_leaf': 10, 'min_samples_split': 2}

In [26]:
dtc_unbalanced = dtc_gs_unbalanced.best_estimator_
dtc_unbalanced.fit(features_train, target_train)


predictions_valid = dtc_unbalanced.predict(features_valid)
predicted_proba_valid = dtc_unbalanced.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели решающего дерева:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели решающего дерева: 0.559774964838256
AUC-ROC метрика: 0.7827276356619626


Результат для дерева решений - 0.546, еще меньше, чем у случайного леса. Но метрика AUC-ROC также неплоха - 0.82.

Рассмотрим теперь логистическую регрессию.

In [27]:
model_lr_unbalanced = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1')
model_lr_unbalanced.fit(features_train, target_train)

LogisticRegression(penalty='l1', random_state=12345, solver='liblinear')

In [28]:
predictions_valid = model_lr_unbalanced.predict(features_valid)
predicted_proba_valid = model_lr_unbalanced.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели логистической регрессии:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели логистической регрессии: 0.32830820770519265
AUC-ROC метрика: 0.759809822222491


Результаты модели логистической регрессии самые низкие. Этот алгоритм ищет взаимосвязи между признаками и целевым признаком. Возможно, в данном случае эти связи не такие прямые и очевидные, поэтому логистическая регрессия показывает не очень хорошие результаты. Также сказывается дисбаланс классов.

### Вывод

По итогам исследования результаты меры F1 у трех моделей получились не очень высокими:
- случайный лес 0.555
- дерево решений 0.559
- логистическая регрессия 0.328

Метрика AUC-ROC:
- случайный лес 0.85
- дерево решений 0.782
- логистическая регрессия 0.76

Для подбора лучших параметров использовался алгоритм GridSearch. Получившаяся модель с этими параметрами обучалась и давала предсказания, которые сравнивались с действительными значениями целевого признака с помощью метрики F1. 

Очевидно, это вызвано тем, что в данных есть сильный дисбаланс классов (80% : 20%). По умолчанию все классы имеют один вес, а так как классы в этом датасете в не ровном соотношении, то без корректировки это приводит к дисбалансу и неправильному обучению.

## Борьба с дисбалансом

### Upsampling

Стоит разобраться с дисбалансов классов. Для этого можно использовать метод upsampling, который копирует данные с меньшим классом несколько раз. В данном случае умножим выборку в 4 раза, чтобы классы стали равнозначны.

In [29]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

Размеры классов до преобразования:

In [30]:
target_train.value_counts()

0    4804
1    1196
Name: exited, dtype: int64

In [31]:
features_train_up, target_train_up = upsample(features_train, target_train, 4)

Размеры классов после преобразования:

In [32]:
target_train_up.value_counts()

0    4804
1    4784
Name: exited, dtype: int64

Итак, разобрались с дисбалансом классов, получилось соотношение примерно 1:1. Теперь можно обучать модели на новой обучающей выборке.

In [33]:
%%time

rfc_up = RandomForestClassifier(random_state=12345)

rfc_gs_up = GridSearchCV(estimator = rfc_up, param_grid = parameters_forest, cv=5, scoring='f1', n_jobs = -1)
rfc_gs_up.fit(features_train_up, target_train_up)
rfc_gs_up.best_params_

CPU times: total: 52.4 s
Wall time: 9min 46s


{'max_depth': 14,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 71}

In [34]:
rfc_up = rfc_gs_up.best_estimator_
rfc_up.fit(features_train_up, target_train_up)

predictions_valid = rfc_up.predict(features_valid)
predicted_proba_valid = rfc_up.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели случайного леса:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели случайного леса: 0.6245772266065388
AUC-ROC метрика: 0.851828888391534


In [35]:
%%time

dtc_up = DecisionTreeClassifier(random_state=12345)

dtc_gs_up = GridSearchCV(estimator = dtc_up, param_grid = parameters_tree, cv=5, scoring='f1', n_jobs = -1)

dtc_gs_up.fit(features_train_up, target_train_up)

dtc_gs_up.best_params_

CPU times: total: 4.23 s
Wall time: 11.8 s


{'max_depth': 14, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [36]:
dtc_up = dtc_gs_up.best_estimator_
dtc_up.fit(features_train_up, target_train_up)

predictions_valid = dtc_up.predict(features_valid)
predicted_proba_valid = dtc_up.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели решающего дерева:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели решающего дерева: 0.5063829787234042
AUC-ROC метрика: 0.7128611956278467


In [37]:
model_lr_up = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1')
model_lr_up.fit(features_train_up, target_train_up)

LogisticRegression(penalty='l1', random_state=12345, solver='liblinear')

In [38]:
predictions_valid = model_lr_up.predict(features_valid)
predicted_proba_valid = model_lr_up.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели логистической регрессии:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели логистической регрессии: 0.48835202761000857
AUC-ROC метрика: 0.7643419691626491


### Вывод

Результаты тестов F1 и AUC-ROC определенно выросли для случайного леса и логистической регрессии, так как мы увеличили размер обучающей выборки, а повторение способствует обучению, поэтому качество обучения моделей повысилось. Также мы избавились от дисбаланса классов, что, конечно же, тоже повышает качество обучения. Но вот результаты дерева решений внезапно снизились, возможно, из-за переобучения. Стоит проверить остальные способы решения проблемы дисбаланса и сравнить результаты.

Случайный лес:
- F1 = 0.624
- AUC-ROC = 0.851

Решающее дерево:
- F1 = 0.506
- AUC-ROC = 0.712

Логистическая регрессия: 
- F1 = 0.488
- AUC-ROC = 0.764

### Downsampling

Также для решения проблемы дисбаланса классов есть техника балансирования классов downsampling, которая заключается
в уменьшении числа объектов большего класса путём случайного удаления объектов большего класса. 

In [39]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

Размеры классов до преобразования:

In [40]:
target_train.value_counts()

0    4804
1    1196
Name: exited, dtype: int64

In [41]:
features_train_down, target_train_down = downsample(features_train, target_train, 0.25)

Размеры классов до преобразования:

In [42]:
target_train_down.value_counts()

0    1201
1    1196
Name: exited, dtype: int64

Теперь соотношение классов в обучающей выборке примерно равно 1:1. Посмотрим, как это повлияет на обучение.

In [43]:
%%time

rfc_down = RandomForestClassifier(random_state=12345)

rfc_gs_down = GridSearchCV(estimator = rfc_down, param_grid = parameters_forest, cv=5, scoring='f1', n_jobs = -1)
rfc_gs_down.fit(features_train_down, target_train_down)
rfc_gs_down.best_params_

CPU times: total: 21.5 s
Wall time: 3min 53s


{'max_depth': 10,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 51}

In [44]:
rfc_down = rfc_gs_down.best_estimator_
rfc_down.fit(features_train_down, target_train_down)

predictions_valid = rfc_down.predict(features_valid)
predicted_proba_valid = rfc_down.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели случайного леса:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели случайного леса: 0.5975494816211122
AUC-ROC метрика: 0.85031666051694


In [45]:
%%time

dtc_down = DecisionTreeClassifier(random_state=12345)

dtc_gs_down = GridSearchCV(estimator = dtc_down, param_grid = parameters_tree, cv=5, scoring='f1', n_jobs = -1)
dtc_gs_down.fit(features_train_down, target_train_down)
dtc_gs_down.best_params_

CPU times: total: 1.22 s
Wall time: 3.4 s


{'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2}

In [46]:
dtc_down = dtc_gs_down.best_estimator_
dtc_down.fit(features_train_down, target_train_down)

predictions_valid = dtc_down.predict(features_valid)
predicted_proba_valid = dtc_down.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели решающего дерева:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели решающего дерева: 0.5784037558685446
AUC-ROC метрика: 0.8306781132235255


In [47]:
model_lr_down = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1')
model_lr_down.fit(features_train_down, target_train_down)

LogisticRegression(penalty='l1', random_state=12345, solver='liblinear')

In [48]:
predictions_valid = model_lr_down.predict(features_valid)
predicted_proba_valid = model_lr_down.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели логистической регрессии:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели логистической регрессии: 0.48793103448275865
AUC-ROC метрика: 0.7631200890399772


### Вывод

Результат для случайного леса получился ниже чем при использовании предыдущего способа. Это объясняется тем, что метод downsampling уменьшает обучающую выборку, что приводит к уменьшению качества обучения в данном случае, так как исследуемый датасет, а следовательно и обучающая выборка, изначально по объему не слишком большие. 

Но для алгоритма дерева решений заметен значительный прирост качества. Возможно, после использования метода upsampling дерево решений переобучилось, поэтому результаты были ниже. 

Результаты для логистической регрессии почти не изменились.

Случайный лес:
- F1 = 0.597
- AUC-ROC = 0.85

Решающее дерево:
- F1 = 0.578
- AUC-ROC = 0.830

Логистическая регрессия: 
- F1 = 0.487
- AUC-ROC = 0.763

### Взвешивание классов

Используем третий способ решения проблемы дисбаланса классов, а именно корректировка весов классов путем изменения гиперпараметра *class_weight = 'balanced'*. Этот способ изменит веса классов так, чтобы меньший класс имел больший вес (во столько раз больше, во сколько больший класс превосходит меньший).

In [49]:
%%time

rfc_w = RandomForestClassifier(random_state=12345, class_weight='balanced')

rfc_gs_w = GridSearchCV(estimator = rfc_w, param_grid = parameters_forest, cv=5, scoring='f1', n_jobs = -1)
rfc_gs_w.fit(features_train, target_train)
rfc_gs_w.best_params_

CPU times: total: 28.9 s
Wall time: 5min 39s


{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 6,
 'n_estimators': 71}

In [50]:
rfc_w = rfc_gs_w.best_estimator_
rfc_w.fit(features_train, target_train)

predictions_valid = rfc_w.predict(features_valid)
predicted_proba_valid = rfc_w.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели случайного леса:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели случайного леса: 0.6314606741573033
AUC-ROC метрика: 0.8543830412717232


In [51]:
%%time

dtc_w = DecisionTreeClassifier(random_state=12345, class_weight='balanced')

dtc_gs_w = GridSearchCV(estimator = dtc_w, param_grid = parameters_tree, cv=5, scoring='f1', n_jobs = -1)
dtc_gs_w.fit(features_train, target_train)
dtc_gs_w.best_params_

CPU times: total: 1.25 s
Wall time: 6.48 s


{'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2}

In [52]:
dtc_w = dtc_gs_w.best_estimator_
dtc_w.fit(features_train, target_train)

predictions_valid = dtc_w.predict(features_valid)
predicted_proba_valid = dtc_w.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели решающего дерева:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели решающего дерева: 0.5748837209302327
AUC-ROC метрика: 0.8295961141792535


In [53]:
model_lr_w = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1', class_weight='balanced')
model_lr_w.fit(features_train, target_train)

LogisticRegression(class_weight='balanced', penalty='l1', random_state=12345,
                   solver='liblinear')

In [54]:
predictions_valid = model_lr_w.predict(features_valid)
predicted_proba_valid = model_lr_w.predict_proba(features_valid)
predictions_one_valid = predicted_proba_valid[:, 1]

print('F1 мера лучшей модели логистической регрессии:', f1_score(target_valid, predictions_valid))
print('AUC-ROC метрика:', roc_auc_score(target_valid, predictions_one_valid))

F1 мера лучшей модели логистической регрессии: 0.48835202761000857
AUC-ROC метрика: 0.7643752381758903


### Вывод

Итак, третий способ показал хорошие итоговые результаты проверки с помощью метрик качества. Результаты для случайного леса незначительно, но выросли, дерево решений также показало хороший результат (хотя и ниже чем при использовании downsampling), логистическая регрессия показала такие же результаты как и при предыдущих способах.

Случайный лес:
- F1 = 0.631
- AUC-ROC = 0.854

Решающее дерево:
- F1 = 0.574
- AUC-ROC = 0.829

Логистическая регрессия: 
- F1 = 0.488
- AUC-ROC = 0.764

Таким образом, лучшая модель - случайный лес с гиперпараметром *class_weight='balanced'*.

## Тестирование модели

Так как мы уже проверили модель на валидационной выборке, то можно включить ее в обучающую, чтобы повысить качество обучения.

In [55]:
rfc_best = rfc_gs_w.best_estimator_
rfc_best.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, min_samples_split=6, n_estimators=71,
                       random_state=12345)

In [56]:
predictions_test = rfc_best.predict(features_test)
predicted_proba_test = rfc_best.predict_proba(features_test)
predicted_one_test = predicted_proba_test[:, 1]

print('F1 мера лучшей модели случайного леса:', f1_score(target_test, predictions_test))
print('AUC-ROC метрика:', roc_auc_score(target_test, predicted_one_test))

F1 мера лучшей модели случайного леса: 0.6162528216704289
AUC-ROC метрика: 0.8530546223715317


Протестировали полученную модель на тестовой выборке, получили более высокие результаты метрик F1 и AUC-ROC. 

Результаты лучшей модели:
- F1 = 0.616
- AUC-ROC = 0.853

Порог пройден, модель прошла проверку.

## Общий вывод

Итак, в данном проекте был проанализирован датасет с данными о клиентах банка, и на основании этих данных исследована задача прогнозирования того, уйдет ли клиент из этого банка или останется.


**1. Подготовка данных**

В начале работы данные необходимо подготовить, исходный датасет нуждается в преобразованиях для решения нашей задачи. Данные были загружены, создан датафрейм, изучен. Дальнейшие шаги подготовки данных:

- Обработаны пропуски в столбце tenure, заполнены медианным значением
- Удален столбце surname, так как он не нужен для решения поставленной задачи
- Категориальные признаки превращены в численные с помощью прямого кодирования OHE
- Датасет разделен на обучающую, валидационную и тестовую выборки
- Столбцы с количественными признаками масштабированы, чтобы избежать того, что модель выделит какие-то признаки как более важные

**2. Исследование моделей с дисбалансом.**

Как выяснилось, в данных присутствовал сильный дисбаланс классов. Для качественного обучения моделей классы должны быть в соотношении примерно 1:1, в наших же данных соотношение было примерно 4:1. 

Для начала при обучении моделей использовался датасет без учета дисбаланса классов. Для подбора лучших параметров использовался алгоритм GridSearch. Получившаяся модель с этими параметрами обучалась и давала предсказания, которые сравнивались с действительными значениями целевого признака с помощью метрики F1 (метрика качества классификации, являющаяся средним гармоническим полноты и точности). 

Также исследовали модели с помощью метрики AUC-ROC - метрика качества классификации, равная площади под ROC-кривой (график зависимости доли истинно положительных ответов от доли ложноположительных ответов) (значения метрики изменяются от 0 до 1, AUC-ROC случайной модели равна 0.5).

Итоги исследования:

Случайный лес:
- F1 = 0.565
- AUC-ROC = 0.843

Решающее дерево:
- F1 = 0.546
- AUC-ROC = 0.821

Логистическая регрессия: 
- F1 = 0.33
- AUC-ROC = 0.758

Нужно было получить результат F1 по крайней мере 0.59, без решения проблемы дисбаланса этого не достичь. 

**3. Борьба с дисбалансом.**

Чтобы добиться желаемого качества моделей необходимо разобраться с дисбалансом. Для этого есть несколько способов:

**3.1. Upsampling**

Первой рассмотрим upsampling - техника балансирования классов, которая заключается в увеличении числа объектов меньшего класса путём их многократного копирования (в данном случае в 4 раза). После преобразования обучающей выборки получили соотношение классов примерно 1:1, после чего обучили на новой выборке модели.

Также для подбора лучших параметров для обучения на новой выборке используем GridSearch.

Результаты тестов F1 и AUC-ROC для случайного леса и логистической регрессии выросли, ведь мы увеличили размер обучающей выборки. Также мы избавились от дисбаланса классов, что, конечно же, тоже повышает качество обучения. Но в данном случае не для дерева решений, оно показало результаты даже меньше чем с дибалансом. Возможно, модель переобучилась, что привело к падению качества.

Результаты после обучения на преобразованной с помощью upsampling выборке:

Случайный лес:
- F1 = 0.624
- AUC-ROC = 0.851

Решающее дерево:
- F1 = 0.506
- AUC-ROC = 0.712

Логистическая регрессия: 
- F1 = 0.488
- AUC-ROC = 0.764

**3.2. Downsampling**

Downsampling - техника балансирования классов, которая заключается в уменьшении числа объектов большего класса путём случайного удаления объектов большего класса (в данном случае уменьшили в 4 раза).

Результат для случайного леса получился ниже чем при использовании предыдущего способа. Это объясняется тем, что метод downsampling уменьшает обучающую выборку, что приводит к уменьшению качества обучения в данном случае, так как исследуемый датасет, а следовательно и обучающая выборка, изначально по объему не слишком большие. Но для алгоритма дерева решений заметен значительный прирост качества, так как мы избавились от дисбаланса, а модель в данном случае не переобучилась (по крайней мере не так сильно). Результаты для логистической регрессии почти не изменились.

Результаты после обучения на преобразованной с помощью downsampling выборке:

Случайный лес:
- F1 = 0.597
- AUC-ROC = 0.850

Решающее дерево:
- F1 = 0.578
- AUC-ROC = 0.830

Логистическая регрессия: 
- F1 = 0.487
- AUC-ROC = 0.763

**3.3. Взвешивание классов**

Третий способ борьбы с дисбалансом - изменение гиперпараметра *class_weight='balanced*, показал хорошие итоговые результаты проверки с помощью метрик качества. Результаты для случайного леса незначительно, но выросли, дерево решений также показало хороший результат (хотя и ниже чем при использовании downsampling), логистическая регрессия показала такие же результаты как и при предыдущих способах.

Случайный лес:
- F1 = 0.631
- AUC-ROC = 0.854

Решающее дерево:
- F1 = 0.574
- AUC-ROC = 0.829

Логистическая регрессия: 
- F1 = 0.488
- AUC-ROC = 0.764

Таким образом, лучшая модель - случайный лес с измененными весами классов (гиперпараметром *class_weight='balanced'*).

**4. Тестирование лучшей модели.**

Сначала заново обучили нашу лучшую модель на обучающей+валидационной выборке, после чего протестировали на тестовой выборке. В итоге модель показала более высокие результаты.

Результаты лучшей модели:

- F1 = 0.616
- AUC-ROC = 0.853